In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import ipywidgets as wg
from IPython.display import display
from gekko import GEKKO

In [1]:
# run once to install gekko
# pip install gekko

In [3]:
# data
ud = np.array([0,0.1,0.1,0.1,1,1,1,1,1,1,1,1,1,1,1])
yd = np.array([0,0.1,-0.1,0.05,0.2,0.4,0.55, \
               0.65,0.7,0.74,0.77,0.79,0.81, \
               0.8,0.79])

# initialize MHE
# MHE Model
m = GEKKO(remote=False)

m.time = np.linspace(0,14,15)

#Parameters
u = m.MV() #input
K = m.MV(value=0.85, lb=0.5, ub=1.5) #gain
tau = m.FV(value=2.5, lb=1, ub=10) #time constant

#Variables
y = m.CV() #measurement

#Equations
m.Equation(tau * y.dt() == -y + K*u)

#Options
m.options.IMODE = 5 #MHE
m.options.EV_TYPE = 1
m.options.TIME_SHIFT = 0
m.options.SOLVER = 1

# STATUS = 0, optimizer doesn't adjust value
# STATUS = 1, optimizer can adjust
u.STATUS = 0
K.STATUS = 1
tau.STATUS = 0
y.STATUS = 1

# FSTATUS = 0, no measurement
# FSTATUS = 1, measurement used to update model
u.FSTATUS = 1
K.FSTATUS = 0
tau.FSTATUS = 0
y.FSTATUS = 1
y.WMODEL = 0.0 # no weight to prior model predictions
y.WMEAS = 10.0 # weight for matching to measurements
#y.MEAS = 0.79 # last measurement in horizon

# DMAX = maximum movement each cycle
K.DMAX = 1
tau.DMAX = .1

# MEAS_GAP = dead-band for measurement / model mismatch
y.MEAS_GAP = 0.2

# Insert measurements
u.value = ud
y.value = yd

# optimize parameters
m.solve(disp=False)

### Manipulated Variable (MV) Tuning - Gain (K)

* COST = (+) minimize MV, (-) maximize MV 
* DCOST = penalty for MV movement 
* DMAX = maximum that MV can move each cycle 
* MV_STEP_HOR = default 1, higher integers allow MV to move less frequently

### Controlled Variable (CV) Tuning - Measured Output (y)

* MEAS_GAP = no penalty dead-band around measurements
* WMEAS = measurement weight in objective function
* WMODEL = prior model prediction weight in objective function

In [4]:
def mhe_tuning(cost,dcost,dmax,mv_step,meas_gap):
    K.COST = cost
    K.DCOST = dcost
    K.DMAX = dmax
    K.MV_STEP_HOR = mv_step
    y.MEAS_GAP = meas_gap
    y.value = yd
    
    # solve MHE
    m.solve(disp=False)
        
    plt.figure(1,figsize=(10,7))
    ax=plt.subplot(2,1,1)
    ax.grid()
    plt.plot(m.time,u,'b-',label=r'$Input (u)$')
    plt.plot(m.time,K,'r-o',label=r'$Gain (K)$')
    plt.legend(loc='best')
    plt.ylabel('MV')

    ax=plt.subplot(2,1,2)
    ax.grid()
    plt.plot(m.time,y,'r:',linewidth=2,label=r'$y_{pred}$')
    plt.plot(m.time,yd+meas_gap/2.0,'k--',linewidth=2,label=r'$y_{db-hi}$')
    plt.plot(m.time,yd,'kx',linewidth=2,label=r'$y_{meas}$')
    plt.plot(m.time,yd-meas_gap/2.0,'k--',linewidth=2,label=r'$y_{db-lo}$')
    plt.legend(loc='best')
    plt.ylabel('CV')
    plt.xlabel('Time (sec)')
    plt.savefig('test.png')
    
cost_slide = wg.FloatSlider(value=0.0,min=-1.0,max=1.0,step=0.1)
dcost_slide = wg.FloatSlider(value=0.1,min=0.0,max=1.0,step=0.1)
dmax_slide = wg.FloatSlider(value=1.0,min=0.0,max=1.0,step=0.1)
mv_step_slide = wg.FloatSlider(value=1,min=1,max=10,step=1)
meas_gap_slide = wg.FloatSlider(value=0.2,min=0.0,max=1.0,step=0.1)
wg.interact(mhe_tuning, cost=cost_slide, \
            dcost=dcost_slide, dmax=dmax_slide, \
            mv_step=mv_step_slide, meas_gap=meas_gap_slide)

interactive(children=(FloatSlider(value=0.0, description='cost', max=1.0, min=-1.0), FloatSlider(value=0.1, de…

<function __main__.mhe_tuning(cost, dcost, dmax, mv_step, meas_gap)>